In [1]:
import dataclasses
import os
import cdsapi
import datetime
import functools
from google.cloud import storage
from graphcast import (
    autoregressive,
    casting,
    checkpoint,
    data_utils,
    graphcast,
    normalization,
    rollout,
)
import haiku as hk
import jax
import math
import numpy as np
import xarray
import pytz
import scipy
from typing import Dict
from graphcast.solar_radiation import (
    get_toa_incident_solar_radiation,
)

client = cdsapi.Client()

gcs_client = storage.Client.create_anonymous_client()
gcs_bucket = gcs_client.get_bucket("dm_graphcast")

singlelevelfields = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "2m_temperature",
    "geopotential",
    "land_sea_mask",
    "mean_sea_level_pressure",
    "toa_incident_solar_radiation",
    "total_precipitation",
]
pressurelevelfields = [
    "u_component_of_wind",
    "v_component_of_wind",
    "geopotential",
    "specific_humidity",
    "temperature",
    "vertical_velocity",
]
predictionFields = [
    "u_component_of_wind",
    "v_component_of_wind",
    "geopotential",
    "specific_humidity",
    "temperature",
    "vertical_velocity",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "2m_temperature",
    "mean_sea_level_pressure",
    "total_precipitation_6hr",
]
pressure_levels = [   1,    2,    3,    5,    7,   10,   20,   30,   50,   70,  100,  125,
        150,  175,  200,  225,  250,  300,  350,  400,  450,  500,  550,  600,
        650,  700,  750,  775,  800,  825,  850,  875,  900,  925,  950,  975,
       1000]
pi = math.pi
gap = 6
predictions_steps = 4
watts_to_joules = 3600
first_prediction = datetime.datetime(2022, 1, 1, 18, 0)


class AssignCoordinates:

    coordinates = {
        "2m_temperature": ["batch", "lon", "lat", "time"],
        "mean_sea_level_pressure": ["batch", "lon", "lat", "time"],
        "10m_v_component_of_wind": ["batch", "lon", "lat", "time"],
        "10m_u_component_of_wind": ["batch", "lon", "lat", "time"],
        "total_precipitation_6hr": ["batch", "lon", "lat", "time"],
        "temperature": ["batch", "lon", "lat", "level", "time"],
        "geopotential": ["batch", "lon", "lat", "level", "time"],
        "u_component_of_wind": ["batch", "lon", "lat", "level", "time"],
        "v_component_of_wind": ["batch", "lon", "lat", "level", "time"],
        "vertical_velocity": ["batch", "lon", "lat", "level", "time"],
        "specific_humidity": ["batch", "lon", "lat", "level", "time"],
        "toa_incident_solar_radiation": ["batch", "lon", "lat", "time"],
        "year_progress_cos": ["batch", "time"],
        "year_progress_sin": ["batch", "time"],
        "day_progress_cos": ["batch", "lon", "time"],
        "day_progress_sin": ["batch", "lon", "time"],
        "geopotential_at_surface": ["lon", "lat"],
        "land_sea_mask": ["lon", "lat"],
    }



In [39]:
# Getting the single and pressure level values.
def get_surface():
    if not os.path.exists("single-level.nc"):
        client.retrieve(
            "reanalysis-era5-single-levels",
            {
                "product_type": "reanalysis",
                "variable": singlelevelfields,
                "grid": "0.25/0.25",
                "year": [2022],
                "month": [1],
                "day": [1],
                "time": [
                    "01:00",
                    "02:00",
                    "03:00",
                    "04:00",
                    "05:00",
                    "06:00",
                    "07:00",
                    "08:00",
                    "09:00",
                    "10:00",
                    "11:00",
                    "12:00",
                ],
                "format": "netcdf",
            },
            "single-level.nc",
        )
    surface = xarray.open_dataset("single-level.nc")
    surface = surface.rename(
        {
            var: singlelevelfields[i]
            for i, var in enumerate(surface.data_vars)
        }
    )
    surface = surface.rename(
        {"geopotential": "geopotential_at_surface"}
    )

    # Rename axes
    surface = surface.rename(
        {"valid_time": "time", "latitude": "lat", "longitude": "lon"}
    )

    # Calculating the sum of the last 6 hours of rainfall
    surface = surface.sortby("time")
    surface["total_precipitation_6hr"] = (
        surface["total_precipitation"].rolling(time=6).sum()
    )
    surface = surface.drop_vars("total_precipitation")
    surface = trim_time(surface)
    return surface

def get_atmo():
    if not os.path.exists("pressure-level.nc"):
        client.retrieve(
            "reanalysis-era5-pressure-levels",
            {
                "product_type": "reanalysis",
                "variable": pressurelevelfields,
                "grid": "0.25/0.25",
                "year": [2022],
                "month": [1],
                "day": [1],
                "time": ["06:00", "12:00"],
                "pressure_level": pressure_levels,
                "format": "netcdf",
            },
            "pressure-level.nc",
        )
    atmo = xarray.open_dataset("pressure-level.nc")
    atmo = atmo.rename(
        {
            var: pressurelevelfields[i]
            for i, var in enumerate(atmo.data_vars)
        }
    )

    # Rename axes for pressurelevel as well
    atmo = atmo.rename(
        {
            "valid_time": "time",
            "latitude": "lat",
            "longitude": "lon",
            "pressure_level": "level",
        }
    )

    return atmo

def drop_extra_coords(data):
    return data.drop_vars('number').drop_vars("expver")

def fix_time(data):
    # Add "datetime" as an extra coordinate with the same values as "time"
    data = data.expand_dims(dim="batch")

    dt_value = np.expand_dims(data.time.values, axis=0)
    
    # Calculate timedelta since the first datetime
    first_datetime = data.time.values[0]
    data['time'] = (data.time - first_datetime).astype('timedelta64[ns]')
    data = data.assign_coords(datetime=xarray.Variable(["batch", "time"], dt_value))
    
    return data

def trim_time(data):
    # Select every 6th sample along the time axis
    return data.isel(time=slice(5, None, 6))

def modify_coordinates(data: xarray.Dataset):

    for var in list(data.data_vars):
        varArray: xarray.DataArray = data[var]
        nonIndices = list(
            set(list(varArray.coords)).difference(
                set(AssignCoordinates.coordinates[var])
            )
        )
        if "datetime" in nonIndices:
            nonIndices.remove("datetime")
        data[var] = varArray.isel(**{coord: 0 for coord in nonIndices})
    # Add batch dimension of length 1
    return data



def pipeline(data):
    data = drop_extra_coords(data)
    data = fix_time(data)
    data = modify_coordinates(data)
    return data

In [23]:
surface = get_surface()
atmo = get_atmo()
org_data = xarray.merge([surface, atmo])
with open("data/source-era5_date-2022-01-01_res-0.25_levels-37_steps-01.nc", "rb") as f:
    ref_data = xarray.load_dataset(f)
# Select all time steps except the first one
ref_data = ref_data.isel(time=slice(1, None))


In [40]:
data = pipeline(org_data)
data

<xarray.Dataset> Size: 2GB
Dimensions:                       (time: 2, lat: 721, lon: 1440, batch: 1,
                                   level: 37)
Coordinates:
  * time                          (time) timedelta64[ns] 16B 00:00:00 06:00:00
  * lat                           (lat) float64 6kB 90.0 89.75 ... -89.75 -90.0
  * lon                           (lon) float64 12kB 0.0 0.25 ... 359.5 359.8
  * level                         (level) float64 296B 1e+03 975.0 ... 2.0 1.0
    datetime                      (batch, time) datetime64[ns] 16B 2022-01-01...
Dimensions without coordinates: batch
Data variables: (12/14)
    10m_u_component_of_wind       (batch, time, lat, lon) float32 8MB -0.259 ...
    10m_v_component_of_wind       (batch, time, lat, lon) float32 8MB 0.08087...
    2m_temperature                (batch, time, lat, lon) float32 8MB 248.1 ....
    geopotential_at_surface       (batch, lat, lon) float32 4MB 0.3228 ... 2....
    land_sea_mask                 (batch, lat, lon) float32 4MB 0.0 0.0 ... 1.0
    mean_sea_level_pressure       (batch, time, lat, lon) float32 8MB 1.031e+...
    ...                            ...
    u_component_of_wind           (batch, time, level, lat, lon) float32 307MB ...
    v_component_of_wind           (batch, time, level, lat, lon) float32 307MB ...
    geopotential                  (batch, time, level, lat, lon) float32 307MB ...
    specific_humidity             (batch, time, level, lat, lon) float32 307MB ...
    temperature                   (batch, time, level, lat, lon) float32 307MB ...
    vertical_velocity             (batch, time, level, lat, lon) float32 307MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-12T12:02 GRIB to CDM+CF via cfgrib-0.9.1...

In [33]:
ref_data

<xarray.Dataset> Size: 2GB
Dimensions:                       (lon: 1440, lat: 721, level: 37, time: 2,
                                   batch: 1)
Coordinates:
  * lon                           (lon) float32 6kB 0.0 0.25 0.5 ... 359.5 359.8
  * lat                           (lat) float32 3kB -90.0 -89.75 ... 89.75 90.0
  * level                         (level) int32 148B 1 2 3 5 ... 950 975 1000
  * time                          (time) timedelta64[ns] 16B 06:00:00 12:00:00
    datetime                      (batch, time) datetime64[ns] 16B 2022-01-01...
Dimensions without coordinates: batch
Data variables: (12/14)
    geopotential_at_surface       (lat, lon) float32 4MB 2.735e+04 ... -0.07617
    land_sea_mask                 (lat, lon) float32 4MB 1.0 1.0 1.0 ... 0.0 0.0
    2m_temperature                (batch, time, lat, lon) float32 8MB 249.0 ....
    mean_sea_level_pressure       (batch, time, lat, lon) float32 8MB 9.957e+...
    10m_v_component_of_wind       (batch, time, lat, lon) float32 8MB 0.03993...
    10m_u_component_of_wind       (batch, time, lat, lon) float32 8MB 0.5558 ...
    ...                            ...
    temperature                   (batch, time, level, lat, lon) float32 307MB ...
    geopotential                  (batch, time, level, lat, lon) float32 307MB ...
    u_component_of_wind           (batch, time, level, lat, lon) float32 307MB ...
    v_component_of_wind           (batch, time, level, lat, lon) float32 307MB ...
    vertical_velocity             (batch, time, level, lat, lon) float32 307MB ...
    specific_humidity             (batch, time, level, lat, lon) float32 307MB ...

In [37]:
data.geopotential[:10], ref_data.geopotential[:10]

(<xarray.DataArray 'geopotential' (time: 2, level: 37, lat: 721, lon: 1440)> Size: 307MB
 [76829760 values with dtype=float32]
 Coordinates:
   * time     (time) timedelta64[ns] 16B 00:00:00 06:00:00
   * lat      (lat) float64 6kB 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
   * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
   * level    (level) float64 296B 1e+03 975.0 950.0 925.0 ... 5.0 3.0 2.0 1.0
 Attributes: (12/31)
     GRIB_paramId:                             129
     GRIB_dataType:                            an
     GRIB_numberOfPoints:                      1038240
     GRIB_typeOfLevel:                         isobaricInhPa
     GRIB_stepUnits:                           1
     GRIB_stepType:                            instant
     ...                                       ...
     GRIB_shortName:                           z
     GRIB_totalNumber:                         0
     GRIB_units:                               m**2 s**-2
     long_name: 

In [32]:
eval_inputs, eval_targets, eval_forcings = (
    data_utils.extract_inputs_targets_forcings(
        data,
        target_lead_times=slice("6h", f"{predictions_steps*6}h"),
        **dataclasses.asdict(task_config),
    )
)
eval_inputs

<xarray.Dataset> Size: 8MB
Dimensions:                       (time: 0, lat: 721, lon: 1440, level: 37,
                                   batch: 1)
Coordinates:
  * lat                           (lat) float64 6kB 90.0 89.75 ... -89.75 -90.0
  * lon                           (lon) float64 12kB 0.0 0.25 ... 359.5 359.8
  * level                         (level) float64 296B 1.0 2.0 ... 975.0 1e+03
  * time                          (time) timedelta64[ns] 0B 
Dimensions without coordinates: batch
Data variables: (12/18)
    2m_temperature                (time, lat, lon) float32 0B ...
    mean_sea_level_pressure       (time, lat, lon) float32 0B ...
    10m_v_component_of_wind       (time, lat, lon) float32 0B ...
    10m_u_component_of_wind       (time, lat, lon) float32 0B ...
    total_precipitation_6hr       (time, lat, lon) float32 0B 
    temperature                   (time, level, lat, lon) float32 0B ...
    ...                            ...
    year_progress_sin             (batch, time) float32 0B 
    year_progress_cos             (batch, time) float32 0B 
    day_progress_sin              (batch, time, lon) float32 0B 
    day_progress_cos              (batch, time, lon) float32 0B 
    geopotential_at_surface       (lat, lon) float32 4MB ...
    land_sea_mask                 (lat, lon) float32 4MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-12T12:02 GRIB to CDM+CF via cfgrib-0.9.1...